In [10]:
import opm_acquire
import os
import h5py
os.getcwd()

'C:\\Users\\jlab\\Repos\\lfm\\lfm'

In [2]:
from opm_acquire.cams import PVCamera

In [3]:
cam = PVCamera()

In [18]:
cam.sensor_shape

(3200, 3200)

In [6]:
cam.frame_shape

(3200, 3200)

In [7]:
cam.roi

{'y_size': 3200,
 'x_size': 3200,
 'y_offset': 0,
 'x_offset': 0,
 'y_bin': 1,
 'x_bin': 1}

In [8]:
cam.set_roi(y_size = 1600, y_offset=1600, y_bin =4,x_size = 1600, x_offset=800, x_bin =2,) 

In [9]:
cam.roi

{'y_size': 1600,
 'x_size': 1600,
 'y_offset': 1600,
 'x_offset': 800,
 'y_bin': 4,
 'x_bin': 2}

In [10]:
cam.frame_shape

(400, 800)

In [11]:
cam.frame_dtype

'uint8'

In [12]:
cam.frame_dtype = "uint16"

AttributeError: can't set attribute

In [14]:
del cam

In [4]:
cam.set_trigger(False)

In [12]:
p_target = os.getcwd()
fn = os.path.join(p_target, "test_background.h5")
        #fh5 = h5py.File(fn, "w")

# collect background frame

cam.set_trigger(False)
bg_im = cam.acquire_stack(100)[0].mean(0, dtype='float32')
with h5py.File(fn, 'w') as fh5:
    fh5.create_dataset("bg", data=bg_im)

RuntimeError: Driver device IOCTL failed (PL_ERR_DDI_DEVICE_IOCTL_FAILED). IOCTL PD_RECEIVE_IMAGE_DATA failed, Error 234 (More data is available. )

In [21]:
cam.arm()

RuntimeError: Driver device IOCTL failed (PL_ERR_DDI_DEVICE_IOCTL_FAILED). IOCTL PD_RECEIVE_IMAGE_DATA failed, Error 234 (More data is available. )

In [24]:
cam.preview(fifo=False)

RuntimeError: Driver device IOCTL failed (PL_ERR_DDI_DEVICE_IOCTL_FAILED). IOCTL PD_RECEIVE_IMAGE_DATA failed, Error 234 (More data is available. )

In [27]:
del cam

NameError: name 'cam' is not defined

In [13]:
cam.disarm()

In [15]:
cam.close()

AttributeError: 'PVCamera' object has no attribute 'close'